# Loading data to recreate dataset as expected

In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn.functional as F
from typing import List, Union
from tqdm import tqdm_notebook
from torchmetrics.functional import kl_divergence

# Install required packages, get files, setup main dataframe

In [2]:
!pip install einops
!pip install torchmetrics
!pip install gimmemotifs    #can take around 10 minutes
!genomepy install hg38

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# #downloading data
!wget https://www.dropbox.com/s/db6up7c0d4jwdp4/train_all_classifier_WM20220916.csv.gz?dl=2
# #changing name
!mv   train_all_classifier_WM20220916.csv.gz?dl=2 train_all_classifier_WM20220916.csv.gz
# #unpcaking 
!gunzip -d /train_all_classifier_WM20220916.csv.gz?dl=2 train_all_classifier_WM20220916.csv
# #new seqs

--2022-11-17 18:38:02--  https://www.dropbox.com/s/db6up7c0d4jwdp4/train_all_classifier_WM20220916.csv.gz?dl=2
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/db6up7c0d4jwdp4/train_all_classifier_WM20220916.csv.gz [following]
--2022-11-17 18:38:03--  https://www.dropbox.com/s/raw/db6up7c0d4jwdp4/train_all_classifier_WM20220916.csv.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0f8ba6b90c60f4e2d4314ff075.dl.dropboxusercontent.com/cd/0/inline/Bw8tw6AFg11KUnznGCGqnl4z35goHlFpOwHi-8YKbR4Cu-fqsnAyyI9JnWeWZTGl9sXP0aTpadRFrfCftiWQ37ryvjLv86BHCXdU0EA5D_PBQPS0cWj8oo2Aqvmh2NqP7fJTbq1LW6UHSTHilmEAUUgjmRUy9EARrzBDsEUydLNDrg/file# [following]
--2022-11-17 18:38:03--  https://uc0f8ba6b90c60f4e2d4314ff075.dl.dropboxusercontent.com/cd/0/inli

In [6]:
df = pd.read_csv("train_all_classifier_WM20220916.csv", sep="\t")
### is the file that's being read from a standard format?
df.head()
df = df.sample(8000) # Using 4000k cells

#Utility functions

In [7]:
df

,Unnamed: 0,seqname,start,end,DHS_width,summit,total_signal,numsamples,numpeaks,C1,...,C10,C11,C12,C13,C14,C15,C16,raw_sequence,component,proportion
139145,2966381,chr7,13310199,13310400,201,13310290,18.579898,18,18,0.000000,...,0.000000,0.099507,0.000000,0.0,0.000000,0.000000,0.000000,TTTTAACAGATGGTGTCAGAAGCGGGGTTCAAAGTAGAGCTTTAAT...,10,1.000000
81078,2563552,chr5,36385460,36385660,200,36385570,2.765151,3,3,0.000938,...,0.000000,0.000000,0.000000,0.0,0.016145,0.000000,0.000000,TACTTCTGTTTTTCTTGAAATTTTTCCACTGTGACATTCACATGTT...,13,0.945106
112615,3461491,chr9,136627000,136627200,200,136627110,0.520319,1,1,0.000000,...,0.004966,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,AGGCCTGAGGTATTGACACTCCCCGGCTCTGTGTTTGGCTTCTGAT...,9,0.759222
158513,658361,chr11,131760167,131760380,213,131760270,426.781809,133,133,0.000000,...,0.031330,0.000000,0.023743,0.0,0.039326,0.002323,0.000000,AACACTGGAAGTGTTGATAGAGGGAGAAGGTTCTATGGGAGAGAAA...,1,0.445778
153366,2413815,chr4,85566180,85566380,200,85566290,142.145037,102,102,0.000000,...,0.000000,0.000000,0.003514,0.0,0.000000,0.000000,0.000000,TTGTTATGTTGGTTTTCTAACCAAGGTGATAGCTCTATTTTGTAAT...,1,0.487665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24345,2175046,chr3,73820520,73820720,200,73820630,3.141591,6,6,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000061,0.000000,0.000000,AGCCTTAGAGAAGAAGCATTTTAATCACACAATCTCACATGAACTG...,3,0.989288
140728,2082745,chr3,5431260,5431480,220,5431370,6.306538,8,8,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.040202,0.000000,TGTGAAGGAGCCGGCTTGCTTGAGGTTAGGGTGTGCAGAGGGGCAT...,14,0.993263
35854,1464147,chr18,56369840,56370100,260,56369980,2.568450,2,2,0.002420,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000282,0.000000,TTCATGAAAGTGAATTCAGCAACTATAGAACTGCCTCTTTGAAGGC...,0,0.664603
41144,641293,chr11,120336483,120336684,201,120336610,24.827661,14,14,0.021680,...,0.009711,0.000000,0.000000,0.0,0.000000,0.000000,0.001764,GGCCGGACGGGCATCTCCCGCCCCTCGCGGGGAGCGTCGGGGAGGA...,0,0.636569


In [8]:
df['raw_sequence']

139145    TTTTAACAGATGGTGTCAGAAGCGGGGTTCAAAGTAGAGCTTTAAT...
81078     TACTTCTGTTTTTCTTGAAATTTTTCCACTGTGACATTCACATGTT...
112615    AGGCCTGAGGTATTGACACTCCCCGGCTCTGTGTTTGGCTTCTGAT...
158513    AACACTGGAAGTGTTGATAGAGGGAGAAGGTTCTATGGGAGAGAAA...
153366    TTGTTATGTTGGTTTTCTAACCAAGGTGATAGCTCTATTTTGTAAT...
                                ...                        
24345     AGCCTTAGAGAAGAAGCATTTTAATCACACAATCTCACATGAACTG...
140728    TGTGAAGGAGCCGGCTTGCTTGAGGTTAGGGTGTGCAGAGGGGCAT...
35854     TTCATGAAAGTGAATTCAGCAACTATAGAACTGCCTCTTTGAAGGC...
41144     GGCCGGACGGGCATCTCCCGCCCCTCGCGGGGAGCGTCGGGGAGGA...
102503    ATAAAGGGCATTTGTTTCTAATTAAACACCAAAACCCTAAACCTTT...
Name: raw_sequence, Length: 8000, dtype: object

In [9]:
def sequences_to_fasta(sequences: Union[List[str],pd.Series], filename) -> None:
  
    """
    Parameters
    ----------
    'sequences': pd.Dataframe containing the raw sequences to be
    converted into fasta
    'filename': name for the fasta file to be generated 

    Returns
    -------
    None: writes a fasta file to disk
    """

    with open(f'{filename}.fasta', 'a') as f:
    ## This won't work for non DF methods, the apply could be reworked
    ## in case source is a list instead of a DF
    ## else, coerce into DF
      f.write('\n'.join(sequences[['Unnamed: 0', 
                            'raw_sequence', 
                            'component']].apply(lambda x : 
                                                f'>{x[0]}_component_{x[2]}\n{x[1]}', 
                                                axis=1).values.tolist()))

In [10]:
sequences_to_fasta(df, 'fastq_train')

In [11]:
!head fastq_train.fasta

>680475_component_1
TGTTCGACACGCACTTCCTGTCATGGCCTGAACCCATCTCTCAGGTTAAATTTTCAAAGAGCTGTGGTTGAAGAGGGAGTCCATTCAGATGGTTGAGGGGCCTTAGAATTTCATTTTTGGTTTACACTCCCCACACAGCTACATTTCTGCCTGCACATAGCTACATTCCTTCCTCACTATATAACCTCCCAATTTTAGTT
>406016_component_7
TTCTCTGTTGGCTCTGTGGGTTGTCAGTGCCATGGGATTTAGCCTTTTCACAGTACTCCCTTGAGAAGATCGCAGAAAAGGGGAGACAATGATACAGCAAAAGCATTTGGAATAATTGGCCCTCTTCTCACTACACCACAAGCTGAGGTTGAGAGAGAAAAGGCTCAAAAAGAAAAGATGCTGTTTGGAGAGAGACAACC
>1034044_component_12
TTGTTAGCACCGGGGAACTTTGTCCTGTGCTGGCTCTCGGGCCTGCCCACACCCTGTGATGGTCCCTTACTAAACTCCTCTCCGTTGCACCCCCGGAACGCAGCAGCTCTTGTCTGCTGGTGTCTTTAACTCCTCTTTCTTACTGTTTGCCAGGAAAGATGTTAAGGGATTTCCCCTATATAGAAAAAAAAAATATCCTT
>2994538_component_15
GACAGTCTTTCTCTCAGAAATTGACGCAAGGCTGCACTCTATTTACTGGAAGGCTTGCAGGCGGTTGACTGCTAATGTGCTCATATGGAATTTGGGCAACAACACCTCTTGCCCTCTCTAGACTGTCACTAAGTACACAATCCGATGCTTGTCATGCTGACTTGGTTTCCTCTGTTCTACTGAGGACTTGGTTGTCCTTC
>869934_component_8
GCCTCATTTATCAGCCATGCCTTTCACTTACTGGTGAAAAGCTAATATAGCCTTGCTTCTCCCTCTCAGTCAACACTATTTAAGGGACTGAT

In [12]:
def compute_motifs_gimme_from_fasta(fasta_file) -> pd.DataFrame:

    """
    Parameters
    ----------
    'fasta': sequences_to_fasta 

    Returns
    -------
    'motif_df_gimme': pandas.DataFrame with the amount of known motifs 
    identified in the provided sequences, as read from gimmemotifs output
    """
    print ('Computing Motifs....')
    !gimme scan $fasta_file -p  JASPAR2020_vertebrates -g hg38 > fasta_file_motifs.bed
    motif_df_gimme = pd.read_csv('fasta_file_motifs.bed', sep='\t', 
                                       skiprows=5, header=None) ## Skiprows?
    return motif_df_gimme

In [13]:
# df_results_seq_guime_count wil be used to compute the motifs' distances
motif_df = compute_motifs_gimme_from_fasta('fastq_train.fasta')

Computing Motifs....
scanning: 100% 15567/15567 [03:20<00:00, 77.79 sequences/s]  


In [14]:
motif_df

,0,1,2,3,4,5,6,7,8
0,680475_component_1,pfmscan,misc_feature,10,24,10.548134,+,.,"motif_name ""MA0598.3_EHF"" ; motif_instance ""CG..."
1,680475_component_1,pfmscan,misc_feature,10,23,11.547365,-,.,"motif_name ""MA0473.3_ELF1"" ; motif_instance ""C..."
2,680475_component_1,pfmscan,misc_feature,9,22,10.064358,+,.,"motif_name ""MA0640.2_ELF3"" ; motif_instance ""A..."
3,680475_component_1,pfmscan,misc_feature,12,21,7.600658,-,.,"motif_name ""MA0028.2_ELK1"" ; motif_instance ""C..."
4,680475_component_1,pfmscan,misc_feature,12,21,8.163058,-,.,"motif_name ""MA0759.1_ELK3"" ; motif_instance ""C..."
...,...,...,...,...,...,...,...,...,...
178973,2150765_component_1,pfmscan,misc_feature,18,25,8.011637,+,.,"motif_name ""MA0716.1_PRRX1"" ; motif_instance ""..."
178974,2150765_component_1,pfmscan,misc_feature,18,25,7.594289,+,.,"motif_name ""MA0717.1_RAX2"" ; motif_instance ""C..."
178975,2150765_component_1,pfmscan,misc_feature,18,25,7.618557,-,.,"motif_name ""MA0630.1_SHOX"" ; motif_instance ""C..."
178976,2150765_component_1,pfmscan,misc_feature,123,134,7.743549,-,.,"motif_name ""UN0118.1_ELK4"" ; motif_instance ""T..."


In [15]:
def motif_count_gimme(motif_df, return_probability=False) -> pd.DataFrame:

    """
    Parameters
    ----------
    'motif_df': pandas.DataFrame containing previously obtained motifs
    from calling compute_motifs_gimme_from_fasta

    'return_probability': if True, scales 
    the occurrences based on the total number of sequences in input

    Returns
    -------
    'motif_counts' : pd.DataFrame with index 'motif_id_name' and associated
    count value, either integers or probabilities.
    """
    motif_df = motif_df.copy() ## This could be worked into a class upstream
    motif_df['motifs'] = motif_df[8].apply(
        lambda x: x.split( 'motif_name "'    )[1].split('"')[0]   )
    motif_df[0] = motif_df[0].apply(
        lambda x : '_'.join(  x.split('_')[:-1])    )
    ## Groupby found motifs
    motifs = motif_df[[0,'motifs']].drop_duplicates().groupby('motifs')
    ## Add counts up
    motif_counts = motifs.count()
    ## Return relative probabilities if specified in argument
    if return_probability: 
      motif_counts = motif_counts/motif_counts.sum()

    return motif_counts

In [16]:
relative_motif_counts = motif_count_gimme(motif_df,return_probability=True)
print(relative_motif_counts)

                        0
motifs                   
MA0002.2_RUNX1   0.001749
MA0003.4_TFAP2A  0.001341
MA0007.3_Ar      0.000955
MA0009.2_TBXT    0.001078
MA0014.3_PAX5    0.001514
...                   ...
UN0339.1_ZNF708  0.001056
UN0340.1_ZNF75A  0.001531
UN0341.1_ZNF768  0.001279
UN0383.1_Nanog   0.001224
UN0384.1_Zbtb17  0.001503

[882 rows x 1 columns]


In [17]:
relative_motif_counts.sum()

0    1.0
dtype: float64

# Kullback Leibler scoring function

In [21]:
def motif_scoring_KL_divergence(original: Union[List[str],pd.Series], 
                                generated: Union[List[str],pd.Series]) -> torch.Tensor:
  
    """
    DOCSTRING IS CURRENTLY INACCURATE, VALUES INSTEAD OF SEQUENCES ARE BEING PROVIDED
    
    Parameters
    ----------
    'original': List | pandas.Series
      original sequences used to train the model
    'generated': List | pandas.Series
      generated sequences obtained from the model

    Returns
    -------
    kl_div: Float
      The Kullback-Leibler divergence between the original and generated
      sequences' distribution.
    """
    ## Consider typesetting directly to Tensor in output (done for now)

    #### WE MUST FIRST COMPARE WHICH MOTIFS CO-OCCUR!
    #### so another function should be introduced before this one to 
    #### set for equal motifs

    p = torch.tensor([original[0]])
    q = torch.tensor([generated[0]])
    return kl_divergence(p,q)


## Reason why pandas series is being used over dataframe:
#https://stackoverflow.com/questions/26047209/what-is-the-difference-between-a-pandas-series-and-a-single-column-dataframe

## If we want to supply a multi-column dataframe and then add a parsing
## statement within the function this could change. However,
## A wrapper to go from the generated dataset to standalone series 
## and then supply a scoring method would probably be better practice


### extend the torchmetrics class due to its use in pyl
## https://torchmetrics.readthedocs.io/en/stable/pages/lightning.html

In [19]:
motif_scoring_KL_divergence(relative_motif_counts,relative_motif_counts)

tensor(0., dtype=torch.float64)